提取图片特征

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.applications.xception import preprocess_input
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten
import h5py
import os
import shutil

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# 参考：https://keras.io/preprocessing/image/
# 参考：https://github.com/ypwhs/dogs_vs_cats
# 图片生成器
def get_img_xception_feature(batch_size_in=32,img_size_in =(299,299,3)):
    feature_file = "modelfile/xception_bottleneck.h5"
    if(os.path.exists(feature_file)):
        raise Exception("file already exists: \""+ feature_file +"\". If you need to extract features again, remove the file first.") 
    if not os.path.exists("modelfile/"):
        os.mkdir("modelfile/")
    
    x_tensor = Input(img_size_in)
    
    """
    datagen =  ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=0.0,
        width_shift_range=0.0,
        height_shift_range=0.0,
        shear_range=0.0,
        zoom_range=0.0,
        horizontal_flip=False,
        vertical_flip = True,
        rescale=1. / 255,
        data_format='channels_last'  #形如（128,128,3）
    )
    """
    datagen =  ImageDataGenerator(preprocessing_function=preprocess_input)
    
    base_model = applications.Xception(input_tensor = x_tensor,weights='imagenet', include_top=False)
    #GlobalAveragePooling2D:输出(samples,2048); Flatten:输出(samples,204800)，Flatten保存的数据太大了，弃之不用；
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output)) #添加GlobalAveragePooling2D层
    # 验证集暂时不与训练集分开
    # 图片预处理，参考：https://keras-cn.readthedocs.io/en/latest/preprocessing/image/
    train_generator = datagen.flow_from_directory("processed_train_data", img_size_in[:2], shuffle=False, 
                                              batch_size=batch_size_in, class_mode='categorical')
    
    test_generator = datagen.flow_from_directory("processed_test_data", img_size_in[:2], shuffle=False, 
                                             batch_size=batch_size_in, class_mode=None)
    # 参考 https://keras.io/models/model/
    train_features = model.predict_generator(train_generator, None,verbose=1) #train_generator.samples)
    test_features = model.predict_generator(test_generator, None,verbose=1) #test_generator.samples)
    
    test_file_que = []
    for i, fname in enumerate(test_generator.filenames):
        test_file_que.append(fname.encode('utf8'))
    
    train_file_que =[]
    for i, fname in enumerate(train_generator.filenames):
        train_file_que.append(fname.encode('utf8'))
    
    with h5py.File(feature_file) as h:
        h.create_dataset("train_features", data=train_features)
        h.create_dataset("test_features", data=test_features)
        h.create_dataset("train_label", data=train_generator.classes)
        h.create_dataset("test_file_que",data=test_file_que)
        h.create_dataset("train_file_que",data=train_file_que)

In [3]:
get_img_xception_feature(batch_size_in=32,img_size_in =(299,299,3))

Found 37399 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


ResourceExhaustedError: OOM when allocating tensor with shape[32,3,299,299]
	 [[Node: block1_conv1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 2, 2, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_input_1_0_1/_1409, block1_conv1/kernel/read)]]
	 [[Node: global_average_pooling2d_1/Mean/_1411 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2614_global_average_pooling2d_1/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'block1_conv1/convolution', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globa

In [6]:
#有时候需要用来备份现有文件
oldfile = "modelfile/xception_bottleneck.h5"
newfile = "modelfile/xception_bottleneck_bak.h5"
#os.rename(oldfile,newfile)
#shutil.copy(oldfile,newfile)